## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Оглавление
### 1. [Изучение данных](#1)
#### 1.1. [Общая информация](#1)
#### 1.2. [Анализ категориальных столбцов](#12)
#### 1.3. [Анализ количественных столбцов](#13)
#### 1.4. [Вывод](#14)

### 2. [Предобработка данных](#2)
#### 2.1. [Обработка пропусков и артефактов](#2)
#### 2.2. [Пропуски в 'total_income'](#22)
#### 2.3. [Пропуски в 'days_employed ''](#23)
#### 2.4. [Обработка артефактов](#24)
#### 2.5. [Выводы](#25)
### 3. [Замена типа данных](#3)
#### 3.1. [Вывод](#31)

### 4. [Обработка дубликатов](#4)
#### 4.1. [Вывод](#41)

### 5. [Лемматизация](#5)
#### 5.1. [Вывод](#51)

### 6. [Категоризация данных](#6)
#### 6.1. [Вывод](#61)

### 7. [Ответы на вопросы](#7)

#### 7.1. [Зависимость между наличием детей и возвратом кредита в срок](#7)
#### 7.2. [Зависимость между семейным положением и возвратом кредита в срок](#72)
#### 7.3. [Зависимость между уровнем дохода и возвратом кредита в срок](#73)
#### 7.4. [Зависимость между уровнем дохода и возвратом кредита в срок](#74)
#### 7.5. [Как разные цели кредита влияют на его возврат в срок](#75)

### 8. [Общий вывод](#8)


<a id="1"></a>

### 1. Изучение данных. 

#### 1.1 Общая информация

In [1]:
import pandas as pd
from IPython.display import display


data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')
data.info()
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<a id="12"></a>

#### 1.2. Анализ категориальных столбцов

Проанализируем "Уровень образования клиента":

In [2]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Видно, что категорий много из-за значительного количества повторов, введенных в разном регистре. Видимо, операционистки в банке не придерживаются общих правил написания (в отличие от нас, аналитиков :) )

In [3]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Посмотрим теперь на "Семейное положение":

In [4]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Немного напрягает, что одно значение написано с большой буквы. Исправляем

In [5]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

В современных реалиях аналитику странно было бы **не** посмотреть что творится в стобце "Пол"

In [6]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Я ожидал увидеть где-то десять уникальных значений :), но в итоге мы обнаружили только одну "аномалию" - "XNA". 
Ввиду того, что сейчас допустимо указывать пол как "Indeterminate/Intersex/Unspecified" данное значение исправлять не стоит, скорее всего оно внесено умышленно и система банка предполагает желание у клиента не указывать свой пол

Так, а чем же занимаются клиенты банка. "Тип занятости"

In [7]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

На первый взгляд никаких артефактов или аномалий

Оценим столбец 'purpose'

In [8]:
display(data['purpose'].unique())

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

<a id="13"></a>

#### 1.3. Анализ количественных столбцов

Перейдем к начальному анализу количественных данных. Взглянем на число детей:

In [9]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

47 записей со значением -1 и это, скорее всего, ошибка, которую нужно будет в дальнейшем исправить, также имеются артефакты в виде 20 детей, потребуется анализ

Далее выведем "Общий трудовой стаж в днях". Т.к. мы уже поняли, что в данном поле у нас есть нули, то посмотрим ненулевые значения.
Начнем с максимумов и минимумов

In [10]:
display(data[data['days_employed'].notnull()].sort_values('days_employed').head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4299,0,-17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
7329,0,-16593.472817,60,высшее,0,женат / замужем,0,F,сотрудник,0,124697.846781,заняться высшим образованием
17838,0,-16264.699501,59,среднее,1,женат / замужем,0,F,сотрудник,0,51238.967133,на покупку автомобиля
16825,0,-16119.687737,64,среднее,1,женат / замужем,0,F,сотрудник,0,91527.685995,покупка жилой недвижимости


In [11]:
data[data['days_employed'].notnull()].sort_values('days_employed').tail(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
10006,0,401715.811749,69,высшее,0,не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью


Есть отрицательные и положительные значения, плюс пустые значения. Тут точно придется поработать

Таким же образом проверим "Ежемесячный доход":

In [12]:
data[data['total_income'].notnull()].sort_values('total_income').head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14585,0,359219.059341,57,среднее,1,женат / замужем,0,F,пенсионер,1,20667.263793,недвижимость
13006,0,369708.589113,37,среднее,1,гражданский брак,1,M,пенсионер,0,21205.280566,заняться высшим образованием
16174,1,-3642.820023,52,Среднее,1,женат / замужем,0,M,сотрудник,0,21367.648356,приобретение автомобиля
1598,0,359726.104207,68,среднее,1,гражданский брак,1,M,пенсионер,0,21695.101789,на проведение свадьбы
14276,0,346602.453782,61,среднее,1,женат / замужем,0,F,пенсионер,0,21895.614355,недвижимость


In [13]:
data[data['total_income'].notnull()].sort_values('total_income').tail(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17178,0,-5734.127087,42,высшее,0,гражданский брак,1,M,компаньон,0,1.711309e+06,сыграть свадьбу
20809,0,-4719.273476,61,среднее,1,не женат / не замужем,4,F,сотрудник,0,1.715018e+06,покупка жилья для семьи
9169,1,-5248.554336,35,среднее,1,гражданский брак,1,M,сотрудник,0,1.726276e+06,дополнительное образование
19606,1,-2577.664662,39,высшее,0,женат / замужем,0,M,компаньон,1,2.200852e+06,строительство недвижимости
12412,0,-1477.438114,44,высшее,0,женат / замужем,0,M,компаньон,0,2.265604e+06,ремонт жилью


Проверим поле возраста клиента

In [14]:
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Как видно, есть записи, где возраст равен нулю. Посмотрим на них

In [15]:
data[data['dob_years'] == 0].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль


In [16]:
data[data['dob_years'] == 0].groupby(['income_type', 'family_status' ]).count()

children  days_employed  dob_years  \
income_type family_status                                               
госслужащий в разводе                     2              2          2   
            гражданский брак              2              2          2   
            женат / замужем               2              2          2   
компаньон   в разводе                     1              1          1   
            гражданский брак              7              6          7   
            женат / замужем              10              9         10   
            не женат / не замужем         2              2          2   
пенсионер   в разводе                     2              1          2   
            вдовец / вдова                5              4          5   
            гражданский брак              1              0          1   
            женат / замужем               8              8          8   
            не женат / не замужем         4              4          4   
сотрудник   в разводе                     5              5          5   
            гражданский брак             11             10         11   
            женат / замужем              29             26         29   
            не женат / не замужем        10              9         10   

                                   education  education_id  family_status_id  \
income_type family_status                                                      
госслужащий в разводе                      2             2                 2   
            гражданский брак               2             2                 2   
            женат / замужем                2             2                 2   
компаньон   в разводе                      1             1                 1   
            гражданский брак               7             7                 7   
            женат / замужем               10            10                10   
            не женат / не замужем          2             2                 2   
пенсионер   в разводе                      2             2                 2   
            вдовец / вдова                 5             5                 5   
            гражданский брак               1             1                 1   
            женат / замужем                8             8                 8   
            не женат / не замужем          4             4                 4   
сотрудник   в разводе                      5             5                 5   
            гражданский брак              11            11                11   
            женат / замужем               29            29                29   
            не женат / не замужем         10            10                10   

                                   gender  debt  total_income  purpose  
income_type family_status                                               
госслужащий в разводе                   2     2             2        2  
            гражданский брак            2     2             2        2  
            женат / замужем             2     2             2        2  
компаньон   в разводе                   1     1             1        1  
            гражданский брак            7     7             6        7  
            женат / замужем            10    10             9       10  
            не женат / не замужем       2     2             2        2  
пенсионер   в разводе                   2     2             1        2  
            вдовец / вдова              5     5             4        5  
            гражданский брак            1     1             0        1  
            женат / замужем             8     8             8        8  
            не женат / не замужем       4     4             4        4  
сотрудник   в разводе                   5     5             5        5  
            гражданский брак           11    11            10       11  
            женат / замужем            29    29            26       29  
            не женат / не замужем     

<a id="14"></a>

#### 1.4. Вывод

В целом данные более-менее понятны.
Есть несколько замечаний:
- У нас есть пропуски, причем одинаковое количество в двух столбцах "days_employed" и "total_income". Тут явная закономерность. Нужно разобраться.
- Столбец "days_employed" представлен пока в непонятном нам формате. С этим надо что-то делать
- Столбец "education" имеет дубликаты из-за разного регистра символов значений
- В столбце "gender" есть значение XNA, которое мы за аномалию счтать не будем
- Столбец "children" однозначно имеет аномальные значения 20 и -1
- В столбце 'dob_years' есть артефакты в виде значения 0
- Столбец 'purpose' определенно имеет схожие значения, записанные по-разному

<a id="2"></a>

### Шаг 2. Предобработка данных

#### 2.1. Обработка пропусков

Из info() мы видим, что в столбцах 'days_employed' и  'total_income' есть нулевые значения, причем их количество совпадает. Попробуем проанализировать закономерность, вычислив количество пропусков по группе "тип занятости" для нулевых значений в этих столбцах

In [17]:
data[data['days_employed'].isnull()].groupby('income_type')['income_type'].count()

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: income_type, dtype: int64

In [18]:
data[data['total_income'].isnull()].groupby('income_type')['income_type'].count()

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: income_type, dtype: int64

При группировке данных по типу заработка по выборкам нулевых значений обоих столбцов выходят одинаковые данные. Количество строк также совпадает. Значит там, где нет общего стажа в днях, так же и нет сведений о заработке.

<a id="22"></a>

#### 2.2. Пропуски в 'total_income'

In [19]:
none_percent = len(data[data['total_income'].isnull()]) / data.shape[0]
print('Процент строк с пропусками: {:.1%}'.format(none_percent))

Процент строк с пропусками: 10.1%


Это оначает, что пропусками не стоит пренебрегать, т.к. отсутствует значительная часть данных

In [20]:
display(data[data['total_income'].isnull()].groupby('debt').sum())

,children,days_employed,dob_years,education_id,family_status_id,total_income
debt,,,,,,
0,1112,0.0,87824,1588,1950,0.0
1,89,0.0,7032,153,170,0.0


Сюда входят люди как с просрочками так и без. Значит все из выборки совершенно точно имели какой-то доход, т.к. кредит без него просто не дают. Наличие просрочки косвенно подтверждает, что ранее люди уже пользовались услугами банка 

Cначала разберемся с количеством ненулевых и нулевых значений в каждой группе
Посчитаем строки без пропусков:

In [21]:
data[data['total_income'].notnull()].groupby('income_type')['total_income'].count()

income_type
безработный            2
в декрете              1
госслужащий         1312
компаньон           4577
пенсионер           3443
предприниматель        1
сотрудник          10014
студент                1
Name: total_income, dtype: int64

Теперь с пропусками:

In [22]:
data[data['total_income'].isnull()].groupby('income_type')['family_status_id'].count()

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: family_status_id, dtype: int64

Можем сделать промежуточный вывод, что пропуски у нас в основном в категориях, представленных обширнее всего.

Посмотрим теперь в целом на распределение дохода (для тех, у кого он отличен от нуля). Вычислим среднее значение и медиану. Заодно выясним зависит ли доход от пола (я, конечно, понимаю что гендерное равенство и все такое, но у нас есть данные, которые мы обязаны проверить)

In [23]:
data[data['total_income'].notnull()].groupby(['gender','income_type'])['total_income'].median()

gender  income_type    
F       безработный        202722.511368
        в декрете           53829.130729
        госслужащий        136982.489425
        компаньон          160820.776207
        пенсионер          115807.789733
        предприниматель    499163.144947
        сотрудник          130615.610597
M       безработный         59956.991984
        госслужащий        185964.946748
        компаньон          196818.800823
        пенсионер          130739.759955
        сотрудник          162161.177474
        студент             98201.625314
XNA     компаньон          203905.157261
Name: total_income, dtype: float64

In [24]:
data[data['total_income'].notnull()].groupby(['gender','income_type'])['total_income'].mean()

gender  income_type    
F       безработный        202722.511368
        в декрете           53829.130729
        госслужащий        155680.311468
        компаньон          184221.811725
        пенсионер          134038.546168
        предприниматель    499163.144947
        сотрудник          148863.158984
M       безработный         59956.991984
        госслужащий        212726.065675
        компаньон          233027.090448
        пенсионер          150734.228922
        сотрудник          180978.520106
        студент             98201.625314
XNA     компаньон          203905.157261
Name: total_income, dtype: float64

Медиана и среднее значение данных по типам занятости похожи. Гендерное различие, пусть незначительное, но есть. Теперь проверим на максимальные значения

In [25]:
data[data['total_income'].notnull()].groupby('income_type')['total_income'].max()

income_type
безработный        2.027225e+05
в декрете          5.382913e+04
госслужащий        9.104515e+05
компаньон          2.265604e+06
пенсионер          7.351033e+05
предприниматель    4.991631e+05
сотрудник          1.726276e+06
студент            9.820163e+04
Name: total_income, dtype: float64

Видно, что максимальные доходы довольно большие, но в жизни так бывает, что и пенсионер может иметь доход в 735 т.р. Возмем для примера несколько категорий и посмотрим на максимумы и минимумы

In [26]:
data[(data['income_type'] == 'пенсионер') & (data['total_income'].notnull())].sort_values(by = 'total_income', ascending = False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15660,0,342783.405779,64,высшее,0,вдовец / вдова,2,F,пенсионер,0,735103.270167,заняться образованием
6084,0,351717.389895,65,высшее,0,женат / замужем,0,M,пенсионер,0,708927.197617,операции с коммерческой недвижимостью
10725,0,373012.754501,64,высшее,0,гражданский брак,1,F,пенсионер,0,694150.890876,покупка жилья для семьи
1131,0,378612.272926,49,среднее,1,гражданский брак,1,M,пенсионер,0,681300.590458,на проведение свадьбы
1409,0,334060.678873,65,среднее,1,женат / замужем,0,F,пенсионер,0,644077.836518,образование


In [27]:
data[(data['income_type'] == 'пенсионер') & (data['total_income'].notnull())].sort_values(by = 'total_income', ascending = False).tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10881,0,347356.519176,71,среднее,1,женат / замужем,0,M,пенсионер,0,22472.755205,операции со своей недвижимостью
14276,0,346602.453782,61,среднее,1,женат / замужем,0,F,пенсионер,0,21895.614355,недвижимость
1598,0,359726.104207,68,среднее,1,гражданский брак,1,M,пенсионер,0,21695.101789,на проведение свадьбы
13006,0,369708.589113,37,среднее,1,гражданский брак,1,M,пенсионер,0,21205.280566,заняться высшим образованием
14585,0,359219.059341,57,среднее,1,женат / замужем,0,F,пенсионер,1,20667.263793,недвижимость


In [28]:
data[(data['income_type'] == 'компаньон') & (data['total_income'].notnull())].sort_values(by = 'total_income', ascending = False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12412,0,-1477.438114,44,высшее,0,женат / замужем,0,M,компаньон,0,2.265604e+06,ремонт жилью
19606,1,-2577.664662,39,высшее,0,женат / замужем,0,M,компаньон,1,2.200852e+06,строительство недвижимости
17178,0,-5734.127087,42,высшее,0,гражданский брак,1,M,компаньон,0,1.711309e+06,сыграть свадьбу
17503,0,-2285.476482,43,среднее,1,женат / замужем,0,M,компаньон,0,1.597613e+06,операции с недвижимостью
18368,1,-333.935516,41,ВЫСШЕЕ,0,гражданский брак,1,M,компаньон,0,1.551153e+06,свадьба


In [29]:
data[(data['income_type'] == 'компаньон') & (data['total_income'].notnull())].sort_values(by = 'total_income', ascending = False).tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2316,1,-145.967024,41,среднее,1,женат / замужем,0,F,компаньон,0,34946.499050,приобретение автомобиля
12449,0,-2992.071835,53,среднее,1,женат / замужем,0,F,компаньон,0,33810.859377,операции со своей недвижимостью
17354,1,-1793.495672,35,среднее,1,гражданский брак,1,F,компаньон,0,33322.631967,сыграть свадьбу
18594,0,-3168.158999,50,среднее,1,женат / замужем,0,F,компаньон,0,31428.891652,операции с недвижимостью
16597,0,-241.247456,49,Среднее,1,гражданский брак,1,F,компаньон,0,28702.812889,сыграть свадьбу


В принципе все выглядит равномерно и реально

Вполне возможно, что пропуски образовались из за неверной выгрузки из АБС банка, т.к. не коррелируют с самым очевидным столбцом 'income_type'
Значит можно заполнить пропуски медианными значениями в зависимости от типа заработка и пола

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [31]:
data['total_income'].fillna(data.groupby(['income_type', 'gender'])['total_income'].transform('median'), inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21524 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Остался один пропуск, посмотрим на него

In [32]:
data[data['total_income'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости


Выясним почему так произошло:

In [33]:
data[data['income_type'] == 'предприниматель']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости
18697,0,-520.848083,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы


Получается, что у нас всего два предпринимателя, удаляем того у кого неизвестен доход, т.к. это не повлияет на медиану и среднее

In [34]:
data.dropna(subset = ['total_income'], inplace = True)

Проверяем, что данные, не "испортились". Снова вычисляем средние значения и медианы по полу и типу источнику дохода

In [35]:
data[data['total_income'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [36]:
data.groupby(['gender','income_type'])['total_income'].median()

gender  income_type    
F       безработный        202722.511368
        в декрете           53829.130729
        госслужащий        136982.489425
        компаньон          160820.776207
        пенсионер          115807.789733
        предприниматель    499163.144947
        сотрудник          130615.610597
M       безработный         59956.991984
        госслужащий        185964.946748
        компаньон          196818.800823
        пенсионер          130739.759955
        сотрудник          162161.177474
        студент             98201.625314
XNA     компаньон          203905.157261
Name: total_income, dtype: float64

In [37]:
data.groupby(['gender','income_type'])['total_income'].mean()

gender  income_type    
F       безработный        202722.511368
        в декрете           53829.130729
        госслужащий        153730.445482
        компаньон          181828.274466
        пенсионер          132006.492253
        предприниматель    499163.144947
        сотрудник          147004.612389
M       безработный         59956.991984
        госслужащий        210293.236681
        компаньон          229554.011263
        пенсионер          148986.861290
        сотрудник          179181.823660
        студент             98201.625314
XNA     компаньон          203905.157261
Name: total_income, dtype: float64

Все в порядке, в целом картина практически не изменилась

<a id="23"></a>

#### 2.3. Пропуски в 'days_employed '

Теперь требуется понять что и как делать с общим трудовым стажем в днях.
Логично, что трудовой стаж в некоторой степени зависит от возраста и типа занятости. Попытаемся проанализировать с этой стороны

In [38]:
data[data['days_employed'].notnull()].groupby(['dob_years','income_type'])['days_employed'].mean().head(10)

dob_years  income_type
0          госслужащий     -5878.760924
           компаньон       -1788.865798
           пенсионер      362537.515114
           сотрудник       -1907.113148
19         госслужащий      -509.969922
           компаньон        -525.137538
           сотрудник        -810.376486
20         госслужащий      -645.671074
           компаньон        -511.714732
           сотрудник        -777.254312
Name: days_employed, dtype: float64

In [39]:
data[data['days_employed'].notnull()].groupby(['dob_years','income_type'])['days_employed'].mean().tail(10)

dob_years  income_type
71         пенсионер      367124.598209
           сотрудник       -3460.651739
72         компаньон       -6603.920060
           пенсионер      368167.883554
73         компаньон       -3429.205485
           пенсионер      365584.660790
74         компаньон       -1729.632531
           пенсионер      359790.863316
           сотрудник       -6682.867814
75         госслужащий     -1678.969771
Name: days_employed, dtype: float64

In [40]:
data[data['days_employed'].notnull()].groupby('dob_years')['days_employed'].median().head(10)

dob_years
0    -1146.689586
19    -724.492610
20    -674.838979
21    -618.733817
22    -698.101581
23    -690.204208
24    -947.731043
25    -919.199388
26   -1064.650275
27   -1137.678822
Name: days_employed, dtype: float64

In [41]:
data[data['days_employed'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,130739.759955,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,185964.946748,образование
29,0,NaN,63,среднее,1,не женат / не замужем,4,F,пенсионер,0,115807.789733,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,136982.489425,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,115807.789733,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,196818.800823,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,130615.610597,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,160820.776207,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,130615.610597,строительство жилой недвижимости


Закономерность если и есть, то она явно не прослеживается. Формат данных не похож ни на UTC ни на какой другой.
Для требуемых выводов нам этот столбец не нужен, однако мы не можем его просто так выкинут, т.к. строки с пропусками составляюь 10% от данных. Поэтому просто заполним нулями:

In [42]:
data['days_employed'].fillna(0.0, inplace = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null float64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


От пропусков избавились, можно переходить к артефактам

<a id="24"></a>

#### 2.4. Обработка артефактов 

Разберемся со столбцом 'children', еще раз посмотрим, что там не так:

In [43]:
data.groupby(['children'])['children'].count()

children
-1        47
 0     14148
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Попробуем определить для каких групп соответствуют аномальные количества детей

In [44]:
data[data['children'] == -1].groupby(['children', 'family_status', 'gender' ])['children'].count()

children  family_status          gender
-1        в разводе              F          4
          вдовец / вдова         F          4
          гражданский брак       F          5
          женат / замужем        F         20
                                 M          9
          не женат / не замужем  F          2
                                 M          3
Name: children, dtype: int64

In [45]:
data[(data['children'] != -1) & (data['children'] != 20)].groupby(['children', 'family_status', 'gender', 'dob_years' ])['children'].mean().head(10000)

children  family_status    gender  dob_years
0         в разводе        F       0            0
                                   21           0
                                   22           0
                                   24           0
                                   25           0
                                               ..
5         женат / замужем  F       35           5
                                   36           5
                                   37           5
                                   38           5
                           M       59           5
Name: children, Length: 1267, dtype: int64

In [46]:
data[data['children'] == 20].groupby(['children', 'family_status', 'gender' ])['children'].count()

children  family_status          gender
20        в разводе              F          1
                                 M          1
          вдовец / вдова         F          4
          гражданский брак       F          8
                                 M          4
          женат / замужем        F         30
                                 M         19
          не женат / не замужем  F          4
                                 M          5
Name: children, dtype: int64

Больше всего "-1" у замужних женщин, значений "20" в принципе не так много, зависимости нет никакой.
Посмотрим, какой процент этих зачений

In [47]:
children_art_percent = len(data[(data['children'] == 20) | (data['children'] == -1)]) / data.shape[0]
print('Процент артефактов с пропусками: {:.2%}'.format(children_art_percent))

Процент артефактов с пропусками: 0.57%


Т.к. чтобы понять статистическую зависимость количества детей от других данных у нас недостаточно полей (нужно больше "социальной" информации), а столбец 'children' нам нужен для основного вывода проекта, то просто удалим строки с артефактами

In [48]:
data = data[(data['children'] != 20) & (data['children'] != -1)]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21401 entries, 0 to 21524
Data columns (total 12 columns):
children            21401 non-null int64
days_employed       21401 non-null float64
dob_years           21401 non-null int64
education           21401 non-null object
education_id        21401 non-null int64
family_status       21401 non-null object
family_status_id    21401 non-null int64
gender              21401 non-null object
income_type         21401 non-null object
debt                21401 non-null int64
total_income        21401 non-null float64
purpose             21401 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


<a id="25"></a>

#### 2.5. Выводы

- Пропуски в "total_income" хорошо поддаются классификаци и анализу, поэтому мы их заполнили медианами на основе имеющихся у нас полей
- Пропуски "days_employed" остались непонятны(скорее всего выгрузка неверная) и не влияют на выводы, поэтому их просто заполнили нулями
- Артефакты в "children" составляют всего 0,55% от общих данных и не поддаются статиситке, поэтому этими строками можно пренебречь, мы их удалили
- Артефакты 'dob_years' = 0 не влияют на выводы, потому их игнорируем

<a id="3"></a>

### 3. Замена типа данных

Для удобства вычислений переведем значение столбца 'total_income' в целые числа. Тут содержатся избыточные дробные данные

In [49]:
data['total_income'] = data['total_income'].astype('int')
data.info()
    

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21401 entries, 0 to 21524
Data columns (total 12 columns):
children            21401 non-null int64
days_employed       21401 non-null float64
dob_years           21401 non-null int64
education           21401 non-null object
education_id        21401 non-null int64
family_status       21401 non-null object
family_status_id    21401 non-null int64
gender              21401 non-null object
income_type         21401 non-null object
debt                21401 non-null int64
total_income        21401 non-null int64
purpose             21401 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


<a id="31"></a>

#### 3.1. Вывод

Для удобства подсчетов и уменьшения объема данных преобразовали 'total_income' в int. До этого в таблице содержалась избыточная дробная информация

<a id="4"></a>

### 4. Обработка дубликатов

При первичном анализе данных мы обратили внимание на столбец 'education'

In [50]:
data['education'].value_counts()

среднее                13667
высшее                  4697
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Видно, что одни и те же значения записаны в разном регистре. Исправляем

In [51]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15136
высшее                  5236
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

Так лучше. Посмотрим теперь на наличие строк с полным совпадением.

In [52]:
data.duplicated().sum()

71

Теперь оценим их

In [53]:
data[data.duplicated()].head(5)
full_duplicated_percent = data.duplicated().sum() / data.shape[0]
print('Процент полностью дублированных строк: {:.2%}'.format(full_duplicated_percent))

Процент полностью дублированных строк: 0.33%


Т.к. процент полностью дублированных строк маленький, то можно считать это ошибкой выгрузки, куда попали одни и те же клиенты. Просто избавляемся от них

In [54]:
data = data.drop_duplicates().reset_index(drop = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21330 entries, 0 to 21329
Data columns (total 12 columns):
children            21330 non-null int64
days_employed       21330 non-null float64
dob_years           21330 non-null int64
education           21330 non-null object
education_id        21330 non-null int64
family_status       21330 non-null object
family_status_id    21330 non-null int64
gender              21330 non-null object
income_type         21330 non-null object
debt                21330 non-null int64
total_income        21330 non-null int64
purpose             21330 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


<a id="41"></a>

#### 4.1. Вывод

Столбец 'education' содержал дубли из за записи в разном регистре. Это было исправлено приведением к строчным буквам.
В данных встретилось небольшое количество дубликатов по всем столбцам, мы их просто удалили посчитав за ошибку выгрузки. Вполне возможно, что у клиента было по несколько продуктов банка

<a id="5"></a>

### 5. Лемматизация

У нас есть столбец, в котором текстовые данные наиболее разрозненны и требуют упорядочивания - 'purpose'
Еще раз взглянем на него

In [55]:
display(data['purpose'].unique())

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Такие строки, как, например, "получение образования" и "заняться образованием" вероятнее всего значат одно и то же. Попробуем определить цель кредита более конкретно используя лемматизацию.

In [56]:
from pymystem3 import Mystem
m = Mystem()
to_lemmatize = data['purpose'].unique()
lemmas_data = []
lemmas_columns = ['purpose','purpose_lemma']


for row in to_lemmatize:
    lem_arr = m.lemmatize(row)
    lem_counter_arr = lem_arr.copy()
    for element in lem_counter_arr:
        #удаляем из лемм все "мусорные" слова, типа предлогов и пробелов
        if len(element)  < 4:
            lem_arr.remove(element)
    lemmas_data.append([row , lem_arr])

lemmas_df = pd.DataFrame(data = lemmas_data, columns = lemmas_columns)
lemmas_df.sort_values(by = 'purpose_lemma')

,purpose,purpose_lemma
24,автомобиль,[автомобиль]
20,автомобили,[автомобиль]
33,высшее образование,"[высокий, образование]"
2,дополнительное образование,"[дополнительный, образование]"
18,жилье,[жилье]
37,заняться высшим образованием,"[заниматься, высокий, образование]"
21,заняться образованием,"[заниматься, образование]"
12,недвижимость,[недвижимость]
5,образование,[образование]
4,операции с жильем,"[операция, жилье]"


После того, как мы сформировали отдельный DataFrame по леммам. Можно внести их в основной DataFrame

In [57]:
data = data.merge(lemmas_df, on = 'purpose', how='left')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, жилье]"
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, автомобиль]"
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, жилье]"
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, образование]"
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, свадьба]"


Явно видно, что целей на кредит не так уж и много. Напишем функцию, которая определяет по леммам удобоваримую для анализа форму цели взятия кредита и применим ее к нашему основному DataFrame

In [58]:
def get_purpose(row):
    test_lemma = row['purpose_lemma']
    if 'автомобиль' in test_lemma:
        return 'автомобиль'
    if 'образование' in test_lemma:
        return 'образование'
    if 'жилье' in test_lemma:
        return 'жилье'
    if 'недвижимость' in test_lemma:
        return 'недвижимость'
    if 'свадьба' in test_lemma:
        return 'свадьба'
    return row['purpose']

data['purpose_short'] = data.apply(get_purpose, axis = 1)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,purpose_short
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, жилье]",жилье
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, автомобиль]",автомобиль
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, жилье]",жилье
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, образование]",образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, свадьба]",свадьба


<a id="51"></a>

#### 5.1 Вывод

Для того, чтобы упростить статистику по целям кредита, потребовалось лемматизировать стольбец 'purpose' в 'purpose_lemma'
Чтобы не "прогонять" все 20 тыс. строк через медленный процесс лемматизации, сначала сделали отдельный DataFrame по леммам уникальных значений, а потом смерджили его с основным. Причем убрали "мусорные" слова, такие как предлоги, пробелы.
Результат оказался быстрым и эффективным.
Чтобы записать короткую форму в столбец 'purpose_short' сделали отдельную категорирующую функцию и применили ее методом apply ко всем строкам.

<a id="6"></a>

### 6. Категоризация данных

Чтобы категорировать данные еще раз взглянем на общую информацию

In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21330 entries, 0 to 21329
Data columns (total 14 columns):
children            21330 non-null int64
days_employed       21330 non-null float64
dob_years           21330 non-null int64
education           21330 non-null object
education_id        21330 non-null int64
family_status       21330 non-null object
family_status_id    21330 non-null int64
gender              21330 non-null object
income_type         21330 non-null object
debt                21330 non-null int64
total_income        21330 non-null int64
purpose             21330 non-null object
purpose_lemma       21330 non-null object
purpose_short       21330 non-null object
dtypes: float64(1), int64(6), object(7)
memory usage: 2.4+ MB


Видно, что для семейного положения и для образования есть отдельные поля "id". Это позволит провести категоризацию и вынести значения в отдельные таблицы с категориями.
Сделаем это сначала для 'family_status', сразу удалим дубликаты 

In [60]:
data_family_status = data[['family_status_id', 'family_status']]
data_family_status = data_family_status.drop_duplicates().reset_index(drop=True)
data_family_status.head(10)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


Теперь выведем категории из столбца 'education', также сразу удалим дубликаты 

In [61]:
data_education = data[['education_id', 'education']]
data_education = data_education.drop_duplicates().reset_index(drop=True)
data_family_status.head(10)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


<a id="61"></a>

#### 6.1. Вывод

Данные категоризированы по принципу того, что в таблице есть прописанные ID для столбцов 'education' и  'family_status'.
После "вынесения" этих массивов в отдельные таблицы мы избавились от дубликатов и существенно сократили объем основного DataFrame, т.к. после вынесения категорий столбцы 'education' и  'family_status' можно просто удалить

<a id="7"></a>

### 7. Ответы на вопросы

#### 7.1. Зависимость между наличием детей и возвратом кредита в срок

Прсмотрим сначала как распределены должники в зависимост от количества детей

In [74]:
data_pivot = data.pivot_table(index=['children'],  values='debt', aggfunc='sum')
data_pivot

,debt
children,
0,1063
1,444
2,194
3,27
4,4
5,0


Теперь числим всех кредиторов с детьми и без детей по наличию и отсуствию задолженности и посчитаем процент просрочки среди людей с детьми и без них

In [63]:
with_children = data[data['children'] > 0].shape[0]
with_children_no_debt = data[(data['children'] > 0) & (data['debt'] == 0)].shape[0]
with_children_has_debt = data[(data['children'] > 0) & (data['debt'] == 1)].shape[0]

no_children = data[data['children'] == 0].shape[0]
no_children_no_debt = data[(data['children'] == 0) & (data['debt'] == 0)].shape[0]
no_children_has_debt = data[(data['children'] == 0) & (data['debt'] == 1)].shape[0]

print('Процент клиентов с детьми без просрочек: {:.1%}'.format(with_children_no_debt / with_children))
print('Процент клиентов с детьми c просрочкой: {:.1%}'.format(with_children_has_debt / with_children))
print()
print('Процент клиентов без детей без просрочек: {:.1%}'.format(no_children_no_debt / no_children))
print('Процент клиентов без детей с просрочкой: {:.1%}'.format(no_children_has_debt / no_children))



Процент клиентов с детьми без просрочек: 90.8%
Процент клиентов с детьми c просрочкой: 9.2%

Процент клиентов без детей без просрочек: 92.5%
Процент клиентов без детей с просрочкой: 7.5%


**Вывод:** <font color='purple'>явной </font> зависимости между наличием детей и возвратом кредита в срок нет. Процент просрочки у клиентов с детьми и без них существенно не отличается чтобы сделать вывод о наличии зависимости. <font color='purple'> Однако, небольшая тенденция на то, что клиенты с детьми имеют дисциплину возврата хуже, чем группа без детей, имеется. Возможно потребуется анализ большего массива данных

<a id="72"></a>

#### 7.2. Зависимость между семейным положением и возвратом кредита в срок

In [118]:
data_pivot = data.pivot_table(index=['family_status'], values='debt', aggfunc='sum')
data_pivot

,debt
family_status,
в разводе,84
вдовец / вдова,63
гражданский брак,385
женат / замужем,927
не женат / не замужем,273


<font color='purple'> Посчитаем проценты, для этого напишем функцию расчета из сводной таблицы

In [124]:
data_pivot = data.pivot_table(index=['family_status'], columns='debt', values='gender', aggfunc='count')

def calc_percentage(row):
    percentage_debt = row[1] / (row[1] + row[0])
    percentage_no_debt = row[0] / (row[1] + row[0])
    print("Для категории {:} доля людей с просрочками:{:.1%}".format(row.name, percentage_debt))
    print("Для категории положения {:} доля людей без просрочек:{:.1%}".format(row.name, percentage_no_debt))
    print("")

data_pivot.apply(calc_percentage, axis = 1)

Для категории в разводе доля людей с просрочками:7.1%
Для категории положения в разводе доля людей без просрочек:92.9%

Для категории вдовец / вдова доля людей с просрочками:6.6%
Для категории положения вдовец / вдова доля людей без просрочек:93.4%

Для категории гражданский брак доля людей с просрочками:9.3%
Для категории положения гражданский брак доля людей без просрочек:90.7%

Для категории женат / замужем доля людей с просрочками:7.6%
Для категории положения женат / замужем доля людей без просрочек:92.4%

Для категории не женат / не замужем доля людей с просрочками:9.8%
Для категории положения не женат / не замужем доля людей без просрочек:90.2%



family_status
в разводе                None
вдовец / вдова           None
гражданский брак         None
женат / замужем          None
не женат / не замужем    None
dtype: object

**Вывод:** Явной зависимости от семейного положения факт возврата кредита не имеет<font color='purple'>, однако люди в гражданском браке и неженатые / не замужем имеют дисциплину по возврату кредитов хуже, чем остальные категории

<a id="73"></a>

#### 7.3. Зависимость между уровнем дохода и возвратом кредита в срок

Для определения зависимости от уровня дохода попробуем сначала ввести категории

In [65]:
data['total_income'].median()

144118.0

In [66]:
data['total_income'].max()

2265604

In [67]:
data['total_income'].min()

20667

Напишем функцию, которая классифицирует доход и введем такое поле. <font color='purple'> Сначала вычислим квантили для ранжирования категорий зарплаты

In [140]:

import numpy as np
quantile = np.quantile(data['total_income'], q=[0.1,0.5,0.75])
quantile

array([ 78709.4, 144118. , 196818. ])

In [141]:


def set_income_total_category(row):
    test_income = row['total_income']
    if test_income < 78709.4:
        return 'низкий'
    if (test_income >= 78709.4) & (test_income < 144118):
        return 'средний'
    if (test_income >= 144118) & (test_income < 196818):
        return 'высокий'
    if test_income >= 196818:
        return 'топ'


data['income_total_category'] = data.apply(set_income_total_category, axis = 1)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,purpose_short,income_total_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, жилье]",жилье,топ
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, автомобиль]",автомобиль,средний
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, жилье]",жилье,высокий
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, образование]",образование,топ
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, свадьба]",свадьба,высокий


In [131]:
data_pivot = data.pivot_table(index=['income_total_category'], values='debt', aggfunc='sum')
data_pivot

,debt
income_total_category,
высокий,1275
низкий,2
средний,99
топ,356


Вычислим проценты ранее запрограммированной функцией

In [143]:
data_pivot = data.pivot_table(index=['income_total_category'], columns='debt', values='gender', aggfunc='count')
data_pivot.apply(calc_percentage, axis = 1)

Для категории высокий доля людей с просрочками:8.8%
Для категории положения высокий доля людей без просрочек:91.2%

Для категории низкий доля людей с просрочками:7.4%
Для категории положения низкий доля людей без просрочек:92.6%

Для категории средний доля людей с просрочками:8.5%
Для категории положения средний доля людей без просрочек:91.5%

Для категории топ доля людей с просрочками:7.1%
Для категории положения топ доля людей без просрочек:92.9%



income_total_category
высокий    None
низкий     None
средний    None
топ        None
dtype: object

**Вывод:** После категоризации уровней дохода и анализа данных явно видно, что доход принципиально не влияет на возврат кредита

<a id="75"></a>

#### 7.5. Как разные цели кредита влияют на его возврат в срок

In [126]:
data_pivot = data.pivot_table(index=['purpose_short'], values='debt', aggfunc='sum')
data_pivot

,debt
purpose_short,
автомобиль,400
жилье,308
недвижимость,472
образование,369
свадьба,183


<font color='purple'> Посчитаем проценты, для этого воспользуемся уже готовой функцию расчета из сводной таблицы

In [127]:
data_pivot = data.pivot_table(index=['purpose_short'], columns='debt', values='gender', aggfunc='count')
data_pivot.apply(calc_percentage, axis = 1)

Для категории автомобиль доля людей с просрочками:9.3%
Для категории положения автомобиль доля людей без просрочек:90.7%

Для категории жилье доля людей с просрочками:6.9%
Для категории положения жилье доля людей без просрочек:93.1%

Для категории недвижимость доля людей с просрочками:7.5%
Для категории положения недвижимость доля людей без просрочек:92.5%

Для категории образование доля людей с просрочками:9.3%
Для категории положения образование доля людей без просрочек:90.7%

Для категории свадьба доля людей с просрочками:7.9%
Для категории положения свадьба доля людей без просрочек:92.1%



purpose_short
автомобиль      None
жилье           None
недвижимость    None
образование     None
свадьба         None
dtype: object

**Вывод:** по таблице видно, что особой зависимости между целью кредита и верятностью его просрочки нет<font color='purple'>, однако люди берущие кредит на автомобиль и образование имеют дисциплину по возврату кредитов хуже, чем остальные категории

<a id="8"></a>

### 8. Общий вывод

Мы проанализировали предоставленные нам банком данные и дали ответы на требуемые от нас вопросы. 
- Есть ли зависимость между наличием детей и возвратом кредита в срок? **Явная зависимость не прослеживается <font color='purple'> Однако, небольшая тенденция на то, что клиенты с детьми имеют дисциплину возврата хуже, чем группа без детей, имеется. Возможно потребуется анализ большего массива данных**
- Есть ли зависимость между семейным положением и возвратом кредита в срок? **Явная зависимость не прослеживается<font color='purple'>, однако люди в гражданском браке и неженатые / не замужем имеют дисциплину по возврату кредитов хуже, чем остальные категории**
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок? **Явная зависимость не прослеживается**
- Как разные цели кредита влияют на его возврат в срок? **Явная зависимость не прослеживается <font color='purple'>, однако люди берущие кредит на автомобиль и образование имеют дисциплину по возврату кредитов хуже, чем остальные категории**

Можно сделать вывод, что аналитику приходится работать с довольно разрозненными и неидеальными данными. Чтобы достичь ответов на поставленные вопросы сначала нужно привести все к готовому к статистике виду. Как минимум придется:
- изученить общую информацию о данных;
- провести предобработку (удалить пропуски, вычислить артефакты, обработать дубликаты);

Возможно дополнительно придется:
- применить к данным лемматизацию
- категоризировать данные
- применить стэмминг

Благодаря инструментам, которые предоставляет Python-библиотека Pandas(+другие распространенные библиотеки для узких задач), все это можно сделать довольно быстро и наглядно.